# trainデータの作成

trainデータは以下のようにする
- 過去5回のレースデータを保持
- 過去のデータが1つも無い場合は削除
- 過去のデータが3回未満である場合は0で穴埋め 

### 過去のデータに，他会場でのレースを使用すべきか悩みどころ...
今回は使用しない

In [1]:
import numpy as np
import pandas as pd
#最大表示列数の指定
pd.set_option('display.max_columns', 300)

In [2]:
race_df = pd.read_csv("./csv/cleaned_race_data.csv", sep=",")

In [3]:
horse_df = pd.read_csv("./csv/cleaned_horse_data.csv", sep=",", low_memory=False)

In [4]:
# race
print(race_df.dtypes)
race_df.tail(3)

race_id                     int64
race_round                  int64
weather                    object
ground_status               int64
date                       object
where_racecourse           object
total_horse_number          int64
frame_number_first          int64
horse_number_first          int64
frame_number_second         int64
horse_number_second         int64
frame_number_third          int64
horse_number_third          int64
tansyo                      int64
hukusyo_first               int64
hukusyo_second              int64
hukusyo_third               int64
wakuren                     int64
umaren                      int64
wide_1_2                    int64
wide_1_3                    int64
wide_2_3                    int64
umatan                      int64
renhuku3                    int64
rentan3                     int64
is_obstacle               float64
ground_type                object
is_left_right_straight     object
distance                    int64
weather_rain  

,race_id,race_round,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,is_obstacle,ground_type,is_left_right_straight,distance,weather_rain,weather_snow
49922,202209030410,10,晴,1,2022-06-26 15:01:00,阪神,16,8,16,5,10,7,14,750,230,140,260,1060,1100,470,1460,560,2650,3230,17270,0.0,ダ,右,1400,0,0
49923,202209030411,11,晴,1,2022-06-26 15:40:00,阪神,18,3,6,5,10,4,7,420,180,250,240,2520,2060,780,710,840,3420,5150,25220,0.0,芝,右,2200,0,0
49924,202209030412,12,晴,1,2022-06-26 16:30:00,阪神,16,3,5,7,14,8,15,990,290,1520,660,9570,34170,6890,3370,10540,52400,137920,1016060,0.0,ダ,右,1200,0,0


In [5]:
# horse
horse_df['date'] = pd.to_datetime(horse_df['date'])
print(horse_df.dtypes)
horse_df.head(3)

race_id                        int64
rank                           int64
frame_number                   int64
horse_number                   int64
horse_id                       int64
age                            int64
burden_weight                float64
rider_id                       int64
goal_time                    float64
half_way_rank                float64
last_time                    float64
odds                         float64
popular                      float64
horse_weight                 float64
tamer_id                       int64
owner_id                      object
date                  datetime64[ns]
is_down                        int64
is_senba                       int64
is_mesu                        int64
is_osu                         int64
distance                       int64
avg_velocity                 float64
horse_weight_dif             float64
burden_weight_rate           float64
dtype: object


,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate
0,200806010101,1,1,2,2005102028,3,54.0,733,73.9,1.0,39.2,8.1,4.0,450.0,138,498800,2008-01-05 09:50:00,0,0,1,0,1200,16.238160,-10.0,0.120000
1,200806010101,2,3,5,2005101044,3,54.0,663,73.9,8.5,38.5,13.4,6.0,482.0,1088,163002,2008-01-05 09:50:00,0,0,1,0,1200,16.238160,2.0,0.112033
2,200806010101,3,2,3,2005104156,3,54.0,684,74.2,10.5,38.5,8.2,5.0,458.0,1054,486800,2008-01-05 09:50:00,0,0,1,0,1200,16.172507,4.0,0.117904


In [6]:
# 全レースコース
pd.unique(race_df['where_racecourse'])

array(['中山', '京都', '小倉', '東京', '中京', '阪神', '福島', '新潟', '函館', '札幌'],
      dtype=object)

## raceからの追加

In [7]:
# raceからのmerge
# 'distance'はすでにある
# 'ground_type','is_left_right_straight' は使用しない
race_df_for_merge = race_df[['race_id',"weather",'ground_status','where_racecourse','total_horse_number','is_obstacle','weather_rain','weather_snow']]

merged_horse_df = pd.merge(horse_df, race_df_for_merge, on='race_id')
merged_horse_df = pd.get_dummies(merged_horse_df, columns=["weather"])

In [8]:
merged_horse_df.head(2)

,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,where_racecourse,total_horse_number,is_obstacle,weather_rain,weather_snow,weather_小雨,weather_小雪,weather_晴,weather_曇,weather_雨,weather_雪
0,200806010101,1,1,2,2005102028,3,54.0,733,73.9,1.0,39.2,8.1,4.0,450.0,138,498800,2008-01-05 09:50:00,0,0,1,0,1200,16.23816,-10.0,0.120000,1,中山,16,0.0,0,0,0,0,0,1,0,0
1,200806010101,2,3,5,2005101044,3,54.0,663,73.9,8.5,38.5,13.4,6.0,482.0,1088,163002,2008-01-05 09:50:00,0,0,1,0,1200,16.23816,2.0,0.112033,1,中山,16,0.0,0,0,0,0,0,1,0,0


## 標準化・正規化しやすいように先にfloatへ

In [9]:
merged_horse_df[merged_horse_df.columns[(merged_horse_df.columns != 'race_id')&(merged_horse_df.columns != 'horse_id')&(merged_horse_df.columns != 'date')&(merged_horse_df.columns != 'owner_id')&(merged_horse_df.columns != 'tamer_id')&(merged_horse_df.columns != 'rider_id')&(merged_horse_df.columns != 'where_racecourse')]] = merged_horse_df[merged_horse_df.columns[(merged_horse_df.columns != 'race_id')&(merged_horse_df.columns != 'horse_id')&(merged_horse_df.columns != 'date')&(merged_horse_df.columns != 'owner_id')&(merged_horse_df.columns != 'tamer_id')&(merged_horse_df.columns != 'rider_id')&(merged_horse_df.columns != 'where_racecourse')]].astype(float)

In [10]:
print(merged_horse_df.dtypes)
merged_horse_df.shape

race_id                        int64
rank                         float64
frame_number                 float64
horse_number                 float64
horse_id                       int64
age                          float64
burden_weight                float64
rider_id                       int64
goal_time                    float64
half_way_rank                float64
last_time                    float64
odds                         float64
popular                      float64
horse_weight                 float64
tamer_id                       int64
owner_id                      object
date                  datetime64[ns]
is_down                      float64
is_senba                     float64
is_mesu                      float64
is_osu                       float64
distance                     float64
avg_velocity                 float64
horse_weight_dif             float64
burden_weight_rate           float64
ground_status                float64
where_racecourse              object
t

(711250, 37)

## 季節情報としてsin,cosを入れる
月と日のデータを使用して季節のsin,cos

hourでその日の時刻も追加（単位はhour）
(minute, second は,実際の予測データに含めることが困難なので，学習データには含めない．)

In [12]:
import tqdm

merged_horse_df = merged_horse_df.assign(season_cos = 0.0)
merged_horse_df = merged_horse_df.assign(season_sin = 0.0)
merged_horse_df = merged_horse_df.assign(time_in_the_day = 0)
for index, one_horse_data in tqdm(merged_horse_df.iterrows()):
    date = one_horse_data['date']
    season_float = (int(date.month) + (int(date.day) / 31) - 1 ) / 12
    merged_horse_df["season_cos"][index] = np.cos(2 * np.pi * season_float)
    merged_horse_df["season_sin"][index] = np.sin(2 * np.pi * season_float)
    time_in_the_day = int(date.hour)
    merged_horse_df["time_in_the_day"][index] = time_in_the_day
    
    

c:\Users\koich\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\Users\koich\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\koich\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

# ラベルエンコーディング

In [20]:
from sklearn.preprocessing import LabelEncoder

merged_horse_df['horse_id'] = LabelEncoder().fit_transform(merged_horse_df['horse_id'])
merged_horse_df['tamer_id'] = LabelEncoder().fit_transform(merged_horse_df['tamer_id'])
merged_horse_df['owner_id'] = LabelEncoder().fit_transform(merged_horse_df['owner_id'])
merged_horse_df['rider_id'] = LabelEncoder().fit_transform(merged_horse_df['rider_id'])

merged_horse_df["horse_id"] = merged_horse_df["horse_id"].astype("category")
merged_horse_df["tamer_id"] = merged_horse_df["tamer_id"].astype("category")
merged_horse_df["owner_id"] = merged_horse_df["owner_id"].astype("category")
merged_horse_df["rider_id"] = merged_horse_df["rider_id"].astype("category")


# データをレースごとに分ける

In [21]:
# 全レースコース
unique_racename = pd.unique(merged_horse_df['where_racecourse'])
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for unique, alphabet_name in zip(unique_racename, racecource_name):
    exec("merged_horse_df_{} = merged_horse_df[merged_horse_df['where_racecourse'] == unique].drop(['where_racecourse'], axis=1)".format(alphabet_name))


## 馬の前回までのレース情報をshiftして結合
追加項目
- 前回のレース日時からの差
- rider_idの変化があるか(あるなら1)

In [22]:
# group by したデータフレームに対して行う処理
def make_one_horse_train_data(one_horse_data):
    one_horse_data = one_horse_data.sort_values('date',ascending=False)
    one_horse_data['pre_date_diff'] = one_horse_data['date'].diff(-1).dt.days
    one_horse_data['is_rider_same'] = (one_horse_data['rider_id'].shift(-1) == one_horse_data['rider_id']) * 1.0
    one_horse_data['is_tamer_same'] = (one_horse_data['tamer_id'].shift(-1) == one_horse_data['tamer_id']) * 1.0
    one_horse_data['is_owner_same'] = (one_horse_data['owner_id'].shift(-1) == one_horse_data['owner_id']) * 1.0


    #  数レース分の情報を結合
    result_df = one_horse_data.copy()
    
    for i in range(1,4):
        # i だけ shiftしたものを得る
        # race_id, sexはいらない
        shift_df = one_horse_data.drop(['race_id','is_senba','is_mesu','is_osu', 'tamer_id', 'owner_id', 'horse_id'], axis=1).shift(-i)
        # 目的のレースとどれだけの時間離れているか？
        shift_df['interval_date'] = (one_horse_data['date'] - shift_df['date']).dt.days
        shift_df.drop(['date'], axis=1, inplace=True)
        shift_df.columns = shift_df.columns + "_" + str(i)
        result_df = pd.concat([result_df, shift_df], axis=1)
    return result_df


In [23]:
# test
one_horse_data = merged_horse_df_kyoto[merged_horse_df_kyoto['horse_id'] == 2004110053].copy()
make_one_horse_train_data(one_horse_data).head(3)

,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,is_obstacle,weather_rain,weather_snow,weather_小雨,weather_小雪,weather_晴,weather_曇,weather_雨,weather_雪,season_cos,season_sin,time_in_the_day,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,age_1,burden_weight_1,rider_id_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,is_obstacle_1,weather_rain_1,weather_snow_1,weather_小雨_1,weather_小雪_1,weather_晴_1,weather_曇_1,weather_雨_1,weather_雪_1,season_cos_1,season_sin_1,time_in_the_day_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,age_2,burden_weight_2,rider_id_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,is_obstacle_2,weather_rain_2,weather_snow_2,weather_小雨_2,weather_小雪_2,weather_晴_2,weather_曇_2,weather_雨_2,weather_雪_2,season_cos_2,season_sin_2,time_in_the_day_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,age_3,burden_weight_3,rider_id_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,is_obstacle_3,weather_rain_3,weather_snow_3,weather_小雨_3,weather_小雪_3,weather_晴_3,weather_曇_3,weather_雨_3,weather_雪_3,season_cos_3,season_sin_3,time_in_the_day_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3


In [24]:
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in racecource_name:
    exec("final_df_{} = merged_horse_df_{}.sort_values('date',ascending=False).groupby('horse_id', as_index=False).apply(make_one_horse_train_data)".format(alphabet_name, alphabet_name))
    exec("final_df_{}.shape".format(alphabet_name))

In [25]:
final_df_nakayama.head()

,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,is_obstacle,weather_rain,weather_snow,weather_小雨,weather_小雪,weather_晴,weather_曇,weather_雨,weather_雪,season_cos,season_sin,time_in_the_day,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,age_1,burden_weight_1,rider_id_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,is_obstacle_1,weather_rain_1,weather_snow_1,weather_小雨_1,weather_小雪_1,weather_晴_1,weather_曇_1,weather_雨_1,weather_雪_1,season_cos_1,season_sin_1,time_in_the_day_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,age_2,burden_weight_2,rider_id_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,is_obstacle_2,weather_rain_2,weather_snow_2,weather_小雨_2,weather_小雪_2,weather_晴_2,weather_曇_2,weather_雨_2,weather_雪_2,season_cos_2,season_sin_2,time_in_the_day_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,age_3,burden_weight_3,rider_id_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,is_obstacle_3,weather_rain_3,weather_snow_3,weather_小雨_3,weather_小雪_3,weather_晴_3,weather_曇_3,weather_雨_3,weather_雪_3,season_cos_3,season_sin_3,time_in_the_day_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3
701862,202206030812,13.0,7.0,11.0,56031,6.0,53.0,30,155.7,1.25,38.4,155.5,12.0,486.0,240,2344,2022-04-17 16:25:00,0.0,0.0,0.0,1.0,2500.0,16.056519,4.0,0.109053,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.283206,0.959059,16,42.0,1.0,1.0,1.0,15.0,6.0,12.0,6.0,57.0,30.0,134.6,1.25,39.2,284.9,15.0,482.0,0.0,2200.0,16.344725,-2.0,0.118257,1.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.409820,0.912166,14.0,76.0,1.0,1.0,1.0,42.0,13.0,7.0,11.0,5.0,57.0,30.0,114.5,7.75,40.2,135.2,14.0,484.0,0.0,1800.0,15.720524,4.0,0.117769,4.0,14.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.979530,-0.201299,14.0,258.0,1.0,1.0,1.0,119.0,13.0,6.0,9.0,5.0,57.0,30.0,121.9,13.5,34.7,81.6,12.0,474.0,0.0,2000.0,16.406891,-10.0,0.120253,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.067510,0.997719,15.0,70.0,1.0,1.0,1.0,378.0
701856,202206030812,7.0,7.0,10.0,66106,4.0,55.0,243,154.3,8.00,35.4,4.2,2.0,492.0,281,3140,2022-04-17 16:25:00,0.0,0.0,0.0,1.0,2500.0,16.202203,-6.0,0.111789,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.283206,0.959059,16,85.0,0.0,1.0,1.0,3.0,3.0,3.0,4.0,54.0,114.0,121.6,8.50,34.8,10.2,5.0,498.0,0.0,2000.0,16.447368,2.0,0.108434,1.0,11.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.931753,0.363094,15.0,28.0,0.0,1.0,1.0,85.0,6.0,4.0,4.0,3.0,53.0,243.0,154.5,5.75,35.5,3.3,2.0,496.0,0.0,2500.0,16.181230,4.0,0.106855,4.0,10.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.994869,-0.101168,15.0,91.0,0.0,1.0,1.0,113.0,1.0,6.0,8.0,3.0,54.0,156.0,151.9,4.0,35.5,8.6,4.0,482.0,0.0,2500.0,16.458196,-8.0,0.112033,1.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.101168,-0.994869,15.0,187.0,0.0,1.0,1.0,204.0
701850,202206030812,1.0,1.0,1.0,57343,6.0,57.0,333,152.8,2.75,34.5,3.0,1.0,494.0,266,1935,2022-04-17 16:25:00,0.0,0.0,0.0,1.0,2500.0,16.361257,-4.0,0.115385,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.283206,0.959059,16,85.0,1.0,1.0,1.0,2.0,1.0,1.0,6.0,56.0,333.0,121.4,10.25,34.4,2.2,1.0,498.0,0.0,2000.0,16.474465,-2.0,0.112450,1.0,11.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.931753,0.363094,15.0,124.0,0.0,0.0,1.0,85.0,1.0,6.0,6.0

In [26]:
# 正解ラベル
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in racecource_name:
    exec("final_df_{}['is_tansyo'] = (final_df_{}['rank']<=1.0) * 1".format(alphabet_name, alphabet_name))
    exec("final_df_{}['is_hukusyo'] = (final_df_{}['rank']<=3.0) * 1".format(alphabet_name, alphabet_name))


In [27]:
"""
# もともとのdate情報, race_idはあとで必要なので保持
# 予想したいときのレース情報は
- frame_number
- horse_number
- age
- sex['is_senba','is_mesu','is_osu']
- burden_weight
- horse_weight
- distance
- horse_weight_dif
- ground_status
- total_horse_number
- is_obstacle
- 天気やコースについて
- pre_date_diff
- is_rider_same_1	is_tamer_same_1	is_owner_same_1	interval_date_1

# あるとまずい情報は
- rank
- goal_time
- last_time
- odds
- popular
- is_down
- avg_velocity
- half_way_rank
- is_down

rank は正解ラベルを付けたあとに落とす

"""

racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in racecource_name:
    exec("final_df_{} = final_df_{}.reset_index(drop=True).sort_values('date')".format(alphabet_name, alphabet_name))
    exec("final_df_{} = final_df_{}.drop(['rank', 'goal_time', 'half_way_rank', 'is_down','last_time', 'odds', 'popular', 'avg_velocity'], axis=1)".format(alphabet_name, alphabet_name))


In [28]:
# 1つ前から先のレースデータがnullの馬がいるようなレースだったら削除する
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in racecource_name:
    exec("print('null:' + str(final_df_{}['rank_1'].isnull().sum()) + ' shape:' + str(final_df_{}.shape))".format(alphabet_name, alphabet_name))
    exec("invalid_race = final_df_{}.loc[final_df_{}['rank_1'].isnull(), 'race_id'].unique()".format(alphabet_name, alphabet_name))
    print(invalid_race)
    exec("deleted_df_{} = final_df_{}[~final_df_{}['race_id'].isin(invalid_race)].copy()".format(alphabet_name, alphabet_name, alphabet_name))
    exec("print('null:' + str(deleted_df_{}['rank_1'].isnull().sum()) + ' shape:' + str(deleted_df_{}.shape))".format(alphabet_name, alphabet_name))


null:35872 shape:(102632, 145)
[200806010101 200806010102 200806010103 ... 202206030810 202206030811
 202206030812]
null:0 shape:(9003, 145)
null:29261 shape:(95698, 145)
[200808010101 200808010102 200808010103 ... 202008040806 202008040809
 202008040812]
null:0 shape:(16326, 145)
null:25694 shape:(59458, 145)
[200810010101 200810010102 200810010103 ... 202210020610 202210020611
 202210020612]
null:0 shape:(2190, 145)
null:36446 shape:(110809, 145)
[200805010101 200805010102 200805010103 ... 202205030810 202205030811
 202205030812]
null:0 shape:(12686, 145)
null:30556 shape:(64170, 145)
[200807010101 200807010102 200807010103 ... 202207040410 202207040411
 202207040412]
null:0 shape:(1500, 145)
null:33819 shape:(103309, 145)
[200809010101 200809010102 200809010103 ... 202209030408 202209030409
 202209030412]
null:0 shape:(14255, 145)
null:26486 shape:(48600, 145)
[200803010101 200803010102 200803010103 ... 202203010610 202203010611
 202203010612]
null:0 shape:(431, 145)
null:33604 shap

In [29]:
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

# 0で欠損値を穴埋め
# for alphabet_name in racecource_name:
#     exec("deleted_df_{}.fillna(0, inplace=True)".format(alphabet_name))


# # 欠損値があるデータは削除
# for alphabet_name in racecource_name:
#     exec("print('欠損値削除前のデータ件数: {}'.format(deleted_df_{}.size))".format(alphabet_name))
#     exec("deleted_df_{} = deleted_df_{}.dropna()".format(alphabet_name))
#     exec("print('欠損値削除後のデータ件数: {}'.format(deleted_df_{}.size))".format(alphabet_name))


In [30]:
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']
for alphabet_name in racecource_name:
    exec("deleted_df_{}.isnull().sum()".format(alphabet_name))

In [31]:
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']
for alphabet_name in racecource_name:
    exec("deleted_df_{}.sort_values('date').head()".format(alphabet_name))
    

In [32]:
# データの保存
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']
for alphabet_name in racecource_name:
    exec("deleted_df_{}.sort_values('date').to_csv('csv/final_data_{}.csv', index=False)".format(alphabet_name, alphabet_name))


In [ ]:
import numpy as np
import pandas as pd
#最大表示列数の指定
pd.set_option('display.max_columns', 300)
final_df = pd.read_csv("csv/final_data_nakayama.csv", low_memory=False)

In [ ]:
final_df.head()

,race_id,frame_number,horse_number,age,burden_weight,rider_id,horse_weight,date,is_senba,is_mesu,is_osu,distance,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,is_obstacle,weather_rain,weather_snow,weather_小雨,weather_小雪,weather_晴,weather_曇,weather_雨,weather_雪,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,age_1,burden_weight_1,rider_id_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,is_obstacle_1,weather_rain_1,weather_snow_1,weather_小雨_1,weather_小雪_1,weather_晴_1,weather_曇_1,weather_雨_1,weather_雪_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,age_2,burden_weight_2,rider_id_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,is_obstacle_2,weather_rain_2,weather_snow_2,weather_小雨_2,weather_小雪_2,weather_晴_2,weather_曇_2,weather_雨_2,weather_雪_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,age_3,burden_weight_3,rider_id_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,is_obstacle_3,weather_rain_3,weather_snow_3,weather_小雨_3,weather_小雪_3,weather_晴_3,weather_曇_3,weather_雨_3,weather_雪_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3,tamer_id_106,tamer_id_110,tamer_id_119,tamer_id_131,tamer_id_138,tamer_id_141,tamer_id_180,tamer_id_184,tamer_id_198,tamer_id_201,tamer_id_202,tamer_id_203,tamer_id_208,tamer_id_216,tamer_id_219,tamer_id_221,tamer_id_224,tamer_id_244,tamer_id_248,tamer_id_253,tamer_id_257,tamer_id_261,...,owner_id_948800,owner_id_949006,owner_id_949030,owner_id_949031,owner_id_950006,owner_id_950030,owner_id_950033,owner_id_950800,owner_id_951005,owner_id_951006,owner_id_951033,owner_id_952006,owner_id_952009,owner_id_952030,owner_id_952033,owner_id_952800,owner_id_953030,owner_id_954030,owner_id_954033,owner_id_955006,owner_id_955030,owner_id_955033,owner_id_956002,owner_id_956009,owner_id_956031,owner_id_956033,owner_id_956800,owner_id_95700,owner_id_957007,owner_id_957009,owner_id_957030,owner_id_957033,owner_id_958031,owner_id_958033,owner_id_958800,owner_id_959030,owner_id_959800,owner_id_960006,owner_id_960030,owner_id_96006,owner_id_96007,owner_id_96031,owner_id_961006,owner_id_962006,owner_id_962033,owner_id_963002,owner_id_963031,owner_id_964030,owner_id_965006,owner_id_965007,owner_id_965009,owner_id_965030,owner_id_965031,owner_id_965800,owner_id_966002,owner_id_966006,owner_id_966007,owner_id_966030,owner_id_967030,owner_id_967033,owner_id_968006,owner_id_96803,owner_id_969006,owner_id_969033,owner_id_970006,owner_id_970007,owner_id_970008,owner_id_97002,owner_id_970030,owner_id_970033,owner_id_97007,owner_id_970800,owner_id_971033,owner_id_972002,owner_id_972007,owner_id_972009,owner_id_972030,owner_id_972033,owner_id_972800,owner_id_973006,owner_id_973007,owner_id_973030,owner_id_973800,owner_id_974007,owner_id_974030,owner_id_974033,owner_id_975002,owner_id_975007,owner_id_975009,owner_id_975030,owner_id_976002,owner_id_976007,owner_id_976800,owner_id_97700,owner_id_977005,owner_id_977007,owner_id_977800,owner_id_978007,owner_id_978033,owner_id_978800,owner_id_979007,owner_id_979033,owner_id_980009,owner_id_980033,owner_id_98007,owner_id_98031,owner_id_981002,owner_id_981800,owner_id_982006,owner_id_982033,owner_id_984031,owner_id_984800,owner_id_985001,owner_id_985002,owner_id_985033,owner_id_986002,owner_id_986031,owner_id_986033,owner_id_98700,owner_id_987033,owner_id_987800,owner_id_988002,owner_id_988005,owner_id_98803,owner_id_988033,owner_id_988800,owner_id_

In [ ]:
final_df[final_df["race_id"] == 201908050412]

,race_id,frame_number,horse_number,age,burden_weight,rider_id,horse_weight,date,is_senba,is_mesu,is_osu,distance,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,is_obstacle,weather_rain,weather_snow,weather_小雨,weather_小雪,weather_晴,weather_曇,weather_雨,weather_雪,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,age_1,burden_weight_1,rider_id_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,is_obstacle_1,weather_rain_1,weather_snow_1,weather_小雨_1,weather_小雪_1,weather_晴_1,weather_曇_1,weather_雨_1,weather_雪_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,age_2,burden_weight_2,rider_id_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,is_obstacle_2,weather_rain_2,weather_snow_2,weather_小雨_2,weather_小雪_2,weather_晴_2,weather_曇_2,weather_雨_2,weather_雪_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,age_3,burden_weight_3,rider_id_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,is_obstacle_3,weather_rain_3,weather_snow_3,weather_小雨_3,weather_小雪_3,weather_晴_3,weather_曇_3,weather_雨_3,weather_雪_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3,tamer_id_106,tamer_id_110,tamer_id_119,tamer_id_131,tamer_id_138,tamer_id_141,tamer_id_180,tamer_id_184,tamer_id_198,tamer_id_201,tamer_id_202,tamer_id_203,tamer_id_208,tamer_id_216,tamer_id_219,tamer_id_221,tamer_id_224,tamer_id_244,tamer_id_248,tamer_id_253,tamer_id_257,tamer_id_261,...,owner_id_948800,owner_id_949006,owner_id_949030,owner_id_949031,owner_id_950006,owner_id_950030,owner_id_950033,owner_id_950800,owner_id_951005,owner_id_951006,owner_id_951033,owner_id_952006,owner_id_952009,owner_id_952030,owner_id_952033,owner_id_952800,owner_id_953030,owner_id_954030,owner_id_954033,owner_id_955006,owner_id_955030,owner_id_955033,owner_id_956002,owner_id_956009,owner_id_956031,owner_id_956033,owner_id_956800,owner_id_95700,owner_id_957007,owner_id_957009,owner_id_957030,owner_id_957033,owner_id_958031,owner_id_958033,owner_id_958800,owner_id_959030,owner_id_959800,owner_id_960006,owner_id_960030,owner_id_96006,owner_id_96007,owner_id_96031,owner_id_961006,owner_id_962006,owner_id_962033,owner_id_963002,owner_id_963031,owner_id_964030,owner_id_965006,owner_id_965007,owner_id_965009,owner_id_965030,owner_id_965031,owner_id_965800,owner_id_966002,owner_id_966006,owner_id_966007,owner_id_966030,owner_id_967030,owner_id_967033,owner_id_968006,owner_id_96803,owner_id_969006,owner_id_969033,owner_id_970006,owner_id_970007,owner_id_970008,owner_id_97002,owner_id_970030,owner_id_970033,owner_id_97007,owner_id_970800,owner_id_971033,owner_id_972002,owner_id_972007,owner_id_972009,owner_id_972030,owner_id_972033,owner_id_972800,owner_id_973006,owner_id_973007,owner_id_973030,owner_id_973800,owner_id_974007,owner_id_974030,owner_id_974033,owner_id_975002,owner_id_975007,owner_id_975009,owner_id_975030,owner_id_976002,owner_id_976007,owner_id_976800,owner_id_97700,owner_id_977005,owner_id_977007,owner_id_977800,owner_id_978007,owner_id_978033,owner_id_978800,owner_id_979007,owner_id_979033,owner_id_980009,owner_id_980033,owner_id_98007,owner_id_98031,owner_id_981002,owner_id_981800,owner_id_982006,owner_id_982033,owner_id_984031,owner_id_984800,owner_id_985001,owner_id_985002,owner_id_985033,owner_id_986002,owner_id_986031,owner_id_986033,owner_id_98700,owner_id_987033,owner_id_987800,owner_id_988002,owner_id_988005,owner_id_98803,owner_id_988033,owner_id_988800,owner_id_